In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [ ]:
#We fectch historical stock data
tickers_cryptos = [
    "ETH-USD",     # Ethereum
    "LINK-USD"     # Chainlink
]


tickers_stocks_etfs = [
    "AMD",         # Advanced Micro Devices
    "PSTG",        # Pure Storage
    "GDXJ",        # VanEck Junior Gold Miners ETF
    "GLD"          # SPDR Gold Shares ETF
]

ticker_alab = "ALAB"  # Astera lab

#Include cash as an asset with zero return and zero volatility
risk_free_rate = 0.01  # 1% annual risk-free rate
# Download historical data

data_alab = yf.download(ticker_alab, start="2024-03-30", end="2025-09-01", progress=False, auto_adjust=False,)
data_alab_cleaned = data_alab['Adj Close'].dropna()

data_stocks_etfs = yf.download(tickers_stocks_etfs, start="2020-01-01", end="2025-09-01", progress=False, auto_adjust=False)
data_stocks_etfs_cleaned = data_stocks_etfs['Adj Close'].dropna()

data_cryptos = yf.download(tickers_cryptos, start="2020-01-01", end="2025-09-01", progress=False, auto_adjust=False)
data_cryptos_cleaned = data_cryptos['Adj Close'].dropna()

# Combine all data into a single DataFrame
all_data = pd.concat([data_alab_cleaned, data_stocks_etfs_cleaned, data_cryptos_cleaned], axis=1)
all_data.columns = [ticker_alab] + tickers_stocks_etfs + tickers_cryptos    
all_data = all_data.dropna()


# Calculate daily returns
daily_returns = all_data.pct_change().dropna()
daily_returns['CASH'] = risk_free_rate / 252  # Assuming 252 trading days in a yearticker_alab = "ALAB"


In [ ]:
#Define the portfolio weights
#This is an example, you can modify the weights as needed
w_alab = 0.1
w_amd = 0.15
w_pstg = 0.15
w_gdxj = 0.15
w_gld = 0.15
w_eth = 0.1
w_link = 0.1
w_cash = 0.1

weights_current = {
    'Weight of ALAB': w_alab,
    'Weight of AMD': w_amd,
    'Weight of PSTG': w_pstg,
    'Weight of GDXJ': w_gdxj,
    'Weight of GLD': w_gld,
    'Weight of ETH': w_eth,
    'Weight of LINK': w_link,
    'Weight of CASH': w_cash
}

total_weight = sum(weights_current.values())

# Validate that the sum of weights is 1.0
if not np.isclose(total_weight, 1.0):
    raise ValueError("The sum of the weights must be 1.0")  

In [ ]:
# Portfolio Optimization Class

# The class encapsulates prices, returns, and optimization methods

class PortfolioOptimizer:
    def __init__(self, returns):
        self.returns = returns
        self.num_assets = returns.shape[1]
        self.mean_returns = returns.mean()
        self.cov_matrix = returns.cov()
    def portfolio_performance(self, weights):
        returns = np.sum(self.mean_returns * weights) * 252
        std_dev = np.sqrt(np.dot(weights.T, np.dot(self.cov_matrix * 252, weights)))
        return returns, std_dev

    def negative_sharpe_ratio(self, weights, risk_free_rate=0.01):
        p_returns, p_std_dev = self.portfolio_performance(weights)
        return -(p_returns - risk_free_rate) / p_std_dev

    def optimize_portfolio(self):
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bounds = tuple((0.01, 0.30) for _ in range(self.num_assets))
        initial_guess = self.num_assets * [1. / self.num_assets,]

        result = minimize(self.negative_sharpe_ratio, initial_guess,
                          method='SLSQP', bounds=bounds,
                          constraints=constraints)

        return result.x


In [ ]:
#Plotting the Efficient Frontier and performing Monte Carlo Simulation
def plot_efficient_frontier(optimizer, num_portfolios=10000):
    results = np.zeros((3, num_portfolios))
    weights_record = []

    for i in range(num_portfolios):
        weights = np.random.random(optimizer.num_assets)
        weights /= np.sum(weights)
        weights_record.append(weights)

        portfolio_return, portfolio_std_dev = optimizer.portfolio_performance(weights)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - 0.01) / portfolio_std_dev  # Sharpe Ratio

    plt.figure(figsize=(10, 7))
    plt.scatter(results[0,:], results[1,:], c=results[2,:], cmap='viridis', marker='o', s=10, alpha=0.3)
    plt.colorbar(label='Sharpe Ratio')
    plt.xlabel('Volatility (Std. Deviation)')
    plt.ylabel('Return')
    plt.title('Efficient Frontier')

    # Highlight the optimal portfolio
    optimal_weights = optimizer.optimize_portfolio()
    opt_return, opt_std_dev = optimizer.portfolio_performance(optimal_weights)
    plt.scatter(opt_std_dev, opt_return, marker='*', color='r', s=500, label='Optimal Portfolio')
    plt.legend()
    plt.show()

In [ ]:
#Running the optimization and plotting
optimizer = PortfolioOptimizer(daily_returns)
plot_efficient_frontier(optimizer)


In [ ]:
optimal_weights = optimizer.optimize_portfolio()

old_weights = pd.Series(
    list(weights_current.values()),  
    index=optimizer.returns.columns,
    name="Current Weight"
)

weights_series = pd.Series(
    optimal_weights,
    index=optimizer.returns.columns,
    name="Optimized Weight"
)

print(f'The old weights were:\n{old_weights}')
print(f'Optimal weights:\n{weights_series}')